In [1]:
import torch
import numpy as np
from utils import *
from dataset import *
from noise2noise import Noise2Noise
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# TODO
# validation batch_size, 
# true_image dataset extraction loop
# image_summary [index] : noise2noise.py : summary.py
config = {
    'device':device,
    'dataset':'gaussian',
    'batch_size_train':12,
    'batch_size_valid':1,
    'num_workers':3,
    'learning_rate':0.001,
    'checkpoint_dir': './models_trained/.',
    'checkpoint_filename': 'latest.pt',
    'root_dir':'.',
    'log_dir': './log/',
    'data_folder':'./data/',
    'num_epochs': 100,
}


In [4]:
trainloader = datasetLoader(config, 'train')
validloader = datasetLoader(config, 'valid')

gaussian dataset was created.
gaussian dataset was created.


In [5]:
NET = Noise2Noise(config, trainloader, validloader,trainable=True)

In [6]:
NET.load_model(config)

In [7]:
NET.loss_size

tensor(3.4170e+08, device='cuda:0', requires_grad=True)

In [ ]:
NET.train(config)

D:\Program_Files\Miniconda3\envs\n2n\lib\site-packages\torch\nn\modules\upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Epoch: 32, total_steps: 0, train_loss: 2932331520.0, time: 0:00:00.300234
Epoch: 32, total_steps: 20, train_loss: 7642713600.0, time: 0:00:00.096513
Epoch: 32, total_steps: 40, train_loss: 9226304256.0, time: 0:00:00.099531
Epoch: 32, total_steps: 60, train_loss: 11049798272.0, time: 0:00:00.097836


In [ ]:
load()

In [ ]:
true_images = trueImages.load_data()
valid_images = validloader.load_data()

In [ ]:
from logger import Logger
from summary import viewSummary
summary = viewSummary(config['log_dir'])
import time

In [ ]:
for epoch,(t,v) in enumerate(zip(true_images, valid_images)):
    
    src, trgt = v
    summary.view_image({'true_images':src.view(-1,3,256,256)[:2].cpu().numpy(),
                                     'output_images':t.view(-1,3,256,256)[:2].detach().cpu().numpy()}, epoch)
    time.sleep(15)

In [ ]:
Net.